<a href="https://colab.research.google.com/github/spicy-shawarma/HDC/blob/main/Capacity_Analysis_of_MAP%2C_HRR%2C_FHRR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
from numpy import dot
from numpy.linalg import norm
import random
from multiprocessing import Pool
import matplotlib.pyplot as plt


MAP, copied here just for completeness. For some reason, no normalization works pretty well in preserving similarity


In [ ]:
def generate_MAP_vector(n):
  vec = np.array([random.choice(([-1, 1])) for _ in range(n)])
  return vec

def MAP_bind(vector1, vector2):
  return np.multiply(vector1, vector2)

def MAP_bundle(*args):

  ret = np.zeros(len(args[0][0]))
  for v in args[0]:

    ret = np.add(ret, v)
    ###ret = np.where(ret >= 0, 1, -1)
  return ret

def MAP_permute(vector1):
  return np.roll(vector1, 1, axis=None)

def MAP_similarity(vector1, vector2):
  return dot(vector1, vector2)/(norm(vector1)*norm(vector2))

Holographic Reduced Representation:


In [ ]:
def generate_HRR_vector(n):
    variance = 1 / n
    std_deviation = np.sqrt(variance)
    vector = np.random.normal(0, std_deviation, n)
    return vector

def bind_HRR(vec1, vec2):
  n = len(vec1)
  bound_vec = np.zeros(n)
  for i in range(n):
    for j in range(n):
      bound_vec[i] += (vec1[j%n] * vec2[(j-i)%n])
  return bound_vec

def bundle_HRR(*args):
  ret = np.zeros(len(args[0][0]))
  for v in args[0]:
    ret = np.add(ret, v)

  l2 = norm(ret)
  if l2 == 0:
     raise ValueError("For some reason, bundled vec is the 0 vector")

  return (ret / l2)

def similarity_HRR(vector1, vector2):
  return dot(vector1, vector2)/(norm(vector1)*norm(vector2))

Fourier Holographic Reduced Representation:

In [ ]:
def generate_FHRR_vector(n):
  v = np.zeros(n, dtype=complex)
  for i in range(n):
    angle = (np.random.uniform((-1 * np.pi), np.pi))
    phasor = np.exp(1j * angle)
    v[i] = phasor/np.sqrt((np.real(phasor)**2) + (np.imag(phasor)**2))
  return v

def bind_FHRR(vec1, vec2):
  return np.multiply(vec1, vec2)

def bundle_FHRR_seq(*args): ###emperical, sequential normalization
  ret = np.zeros(len(args[0][0]), dtype=complex)

  for v in args[0]:
    ret = np.add(ret, v)
    l2 = norm(ret)

    if l2 == 0:
      raise ValueError("For some reason, bundled vec is the 0 vector")

  return (ret / l2)

def bundle_FHRR(*args): ###empirical normalization
  ret = np.zeros(len(args[0][0]), dtype=complex)
  for v in args[0]:
    ret = np.add(ret, v)

  l2 = norm(ret)
  if l2 == 0:
     raise ValueError("For some reason, bundled vec is the 0 vector")

  return (ret / l2)

def sim_FHRR(vec1, vec2):
  return np.real((dot(vec1, np.conjugate(vec2)))) / len(vec1)

Capacity Analysis experiment! This takes FOREVER (Upto 10 hours). Results here are just what you get from one time. Not a whole lot of value in repeating it 10 times as you can fit straight lines. I tried being as optimal as I could with array operations. I also don't know much about Python multiprocessing yet so maybe that's why too. But not about to implement the vector stuff in C so this is what we get!!

If you want to wait for it to run then you'll see that yes, you get the same results qualitatively as the paper. If you run it locally with more cores it's way faster.

I could have also optimized it by guessing a lower bound on the minimum number of dimensions but that struck me as scientifically dishonest so instead we just get this horribly slow code.

The assigment gets the experiment details wrong. Here I implemented the experiment from the paper.

In [ ]:


def capacity_analysis(generate, bundle, similarity, complex_elements, name):
  output = np.zeros(50)
  for i in range(5, len(output)): ### number of vectors we are bundling

    for j in range(5, 10000): ### j is the dimension of the vector space. We will usually break out of this loop much earlier than j = 10,000.

      strike = 0

      if complex_elements:
        item_memory = np.ndarray((1000, j), dtype=complex)

      else:
        item_memory = np.ndarray((1000, j))

      for k in range(1000):
        item_memory[k] = generate(j) ### generate random vector of this architecture of dim j, populating item memory


      for _ in range(100):
        random_indices = np.random.randint(0, 1000, i)
        elementary_vectors = np.array(item_memory[random_indices]) ### vectors we are bundling
        bound_vec = bundle(elementary_vectors) ### bundled vector

        similarities = []
        for idx in range(1000):
          similarities.append(similarity(bound_vec, item_memory[idx]))

        sorted_indices = np.argsort(similarities)[::-1] ### get the indeces sorted in order of similarity to bundled vector

        query_response = item_memory[sorted_indices][:i] ### get the i most similar vectors

        m = (query_response[:, None] == elementary_vectors).all(-1).any(1) ### this is a mask which when applied to query_response,
                                                                          ### gives the intersection of query response and elem vecs. next code cell has an example




        if (len(query_response[m]) < len(elementary_vectors)):
          strike += 1
        if (strike >= 2): ### 2 failed retrievals out of 100 --> less than 99% accuracy
          break



      if (strike <= 1):
        print(name + " with " + str(i) + " bundled vectors. Dimension for 99% accuracy = " + str(j))
        output[i] = j
        break


  return output

VSA_Settings = [(generate_MAP_vector, MAP_bundle, MAP_similarity, False, "MAP"),
                 (generate_HRR_vector, bundle_HRR, similarity_HRR,  False, "HRR"),
                (generate_FHRR_vector, bundle_FHRR, sim_FHRR, True, "FHRR"),
                (generate_FHRR_vector, bundle_FHRR_seq, sim_FHRR, True, "FHRR seq norm")
                 ]
lowest_dimensions = np.zeros((4,50))



pool = Pool()

lowest_dimensions = np.add(lowest_dimensions, pool.starmap(capacity_analysis, VSA_Settings))
pool.close()
pool.join()

###lowest_dimensions[0] = np.add(lowest_dimensions[0], capacity_analysis(generate_MAP_vector, MAP_bundle, MAP_similarity, False, "MAP"))
###lowest_dimensions[1] = np.add(lowest_dimensions[1], capacity_analysis(generate_HRR_vector, bundle_HRR, sim_HRR, False, "HRR"))
###lowest_dimensions[2] = np.add(lowest_dimensions[2], capacity_analysis(generate_FHRR_vector, bundle_FHRR, sim_FHRR, True, "FHRR"))
###lowest_dimensions[3] = np.add(lowest_dimensions[3], capacity_analysis(generate_FHRR_vector, bundle_FHRR_seq, sim_FHRR, True, "FHRR seq norm"))

plt.plot(lowest_dimensions[0], label = "MAP")
plt.plot(lowest_dimensions[1], label = "HRR")
plt.plot(lowest_dimensions[2], label = "FHRR")
plt.plot(lowest_dimensions[3], label = "FHRR_Sequential_Normalization")
plt.show()




















HRR with 5 bundled vectors. Dimension for 99% accuracy = 162
HRR with 6 bundled vectors. Dimension for 99% accuracy = 182


Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 51, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-7b8e62b801d2>", line 26, in capacity_analysis
    similarities.append(similarity(bound_vec, item_memory[idx]))
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 125,

KeyboardInterrupt: ignored

  File "<__array_function__ internals>", line 180, in norm
  File "/usr/local/lib/python3.10/dist-packages/numpy/linalg/linalg.py", line 2512, in norm
    x = x.ravel(order='K')
  File "/usr/local/lib/python3.10/dist-packages/numpy/linalg/linalg.py", line 2513, in norm
    if isComplexType(x.dtype.type):
  File "/usr/local/lib/python3.10/dist-packages/numpy/linalg/linalg.py", line 116, in isComplexType
    def isComplexType(t):
KeyboardInterrupt
KeyboardInterrupt


In [ ]:

A = np.array([[1,4],[2,5],[3,6]])
B = np.array([[1,4],[3,6],[7,8]])

m = (A[:, None] == B).all(-1).any(1)

A[m]